<a href="https://colab.research.google.com/github/SAH-UJA/Machine-Translation/blob/main/Copy_of_Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
 
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'deu.txt'

In [ ]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [ ]:
!wget "http://www.manythings.org/anki/deu-eng.zip"

--2020-10-20 12:51:28--  http://www.manythings.org/anki/deu-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.108.196, 104.24.109.196, 172.67.173.198, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.108.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8324970 (7.9M) [application/zip]
Saving to: ‘deu-eng.zip’

deu-eng.zip         100%[===================>]   7.94M  11.7MB/s    in 0.7s    

2020-10-20 12:51:28 (11.7 MB/s) - ‘deu-eng.zip’ saved [8324970/8324970]



In [ ]:
!unzip deu-eng.zip

Archive:  deu-eng.zip
  inflating: deu.txt                 
  inflating: _about.txt              


In [ ]:
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [ ]:
lines

['Go.\tGeh.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8597805 (Roujin)',
 'Hi.\tHallo!\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #380701 (cburgmer)',
 'Hi.\tGrüß Gott!\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #659813 (Esperantostern)',
 'Run!\tLauf!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #941078 (Fingerhut)',
 'Run.\tLauf!\tCC-BY 2.0 (France) Attribution: tatoeba.org #4008918 (JSakuragi) & #941078 (Fingerhut)',
 'Wow!\tPotzdonner!\tCC-BY 2.0 (France) Attribution: tatoeba.org #52027 (Zifre) & #2122382 (Pfirsichbaeumchen)',
 'Wow!\tDonnerwetter!\tCC-BY 2.0 (France) Attribution: tatoeba.org #52027 (Zifre) & #2122391 (Pfirsichbaeumchen)',
 'Fire!\tFeuer!\tCC-BY 2.0 (France) Attribution: tatoeba.org #1829639 (Spamster) & #1958697 (Tamy)',
 'Help!\tHilfe!\tCC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #575889 (MUIRIEL)',
 'Help!\tZu Hülf!\tCC-BY 2.0 (France) Attribution: tatoeba.org #435084

In [ ]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [ ]:
input_texts

['Go.',
 'Hi.',
 'Hi.',
 'Run!',
 'Run.',
 'Wow!',
 'Wow!',
 'Fire!',
 'Help!',
 'Help!',
 'Stop!',
 'Wait!',
 'Wait.',
 'Begin.',
 'Go on.',
 'Hello!',
 'Hurry!',
 'Hurry!',
 'I hid.',
 'I hid.',
 'I ran.',
 'I see.',
 'I see.',
 'I try.',
 'I won!',
 'I won!',
 'Relax.',
 'Shoot!',
 'Shoot!',
 'Smile.',
 'Ask me.',
 'Ask me.',
 'Ask me.',
 'Attack!',
 'Attack!',
 'Cheers!',
 'Eat it.',
 'Eat up.',
 'Eat up.',
 'Freeze!',
 'Freeze!',
 'Go now.',
 'Got it!',
 'Got it!',
 'Got it!',
 'Got it?',
 'Got it?',
 'Got it?',
 'He ran.',
 'He ran.',
 'Hop in.',
 'Hop in.',
 'Hug me.',
 'Hug me.',
 'Hug me.',
 'I care.',
 'I fell.',
 'I fell.',
 'I fell.',
 'I fell.',
 'I fell.',
 'I fled.',
 'I fled.',
 'I know.',
 'I lied.',
 'I lost.',
 'I paid.',
 'I paid.',
 'I sang.',
 'I spit.',
 'I spit.',
 'I swim.',
 'I wept.',
 'I wept.',
 "I'm 19.",
 "I'm 19.",
 "I'm OK.",
 "I'm OK.",
 "I'm up.",
 "I'm up.",
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'Really?',
 'Really?',
 'Really?

In [ ]:
target_texts

['\tGeh.\n',
 '\tHallo!\n',
 '\tGrüß Gott!\n',
 '\tLauf!\n',
 '\tLauf!\n',
 '\tPotzdonner!\n',
 '\tDonnerwetter!\n',
 '\tFeuer!\n',
 '\tHilfe!\n',
 '\tZu Hülf!\n',
 '\tStopp!\n',
 '\tWarte!\n',
 '\tWarte.\n',
 '\tFang an.\n',
 '\tMach weiter.\n',
 '\tHallo!\n',
 '\tBeeil dich!\n',
 '\tSchnell!\n',
 '\tIch versteckte mich.\n',
 '\tIch habe mich versteckt.\n',
 '\tIch rannte.\n',
 '\tIch verstehe.\n',
 '\tAha.\n',
 '\tIch probiere es.\n',
 '\tIch hab gewonnen!\n',
 '\tIch habe gewonnen!\n',
 '\tEntspann dich.\n',
 '\tFeuer!\n',
 '\tSchieß!\n',
 '\tLächeln!\n',
 '\tFrag mich!\n',
 '\tFragt mich!\n',
 '\tFragen Sie mich!\n',
 '\tAngriff!\n',
 '\tAttacke!\n',
 '\tZum Wohl!\n',
 '\tIss es.\n',
 '\tIss auf.\n',
 '\tIss auf!\n',
 '\tKeine Bewegung!\n',
 '\tStehenbleiben!\n',
 '\tGeh jetzt!\n',
 '\tVerstanden!\n',
 '\tAha!\n',
 '\tIch habs!\n',
 '\tKapiert?\n',
 '\tVerstanden?\n',
 '\tEinverstanden?\n',
 '\tEr rannte.\n',
 '\tEr lief.\n',
 '\tMach mit!\n',
 '\tSpring rein!\n',
 '\tDrück mich!\n

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [ ]:
input_characters

[' ',
 '!',
 '"',
 '$',
 '%',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [ ]:
target_characters

['\t',
 '\n',
 ' ',
 '!',
 '$',
 '%',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 'Ä',
 'Ö',
 'Ü',
 'ß',
 'ä',
 'ö',
 'ü',
 '’',
 '“',
 '„',
 '\u202f']

In [ ]:
num_encoder_tokens

71

In [ ]:
num_decoder_tokens

85

In [ ]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 85
Max sequence length for inputs: 15
Max sequence length for outputs: 45


In [ ]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [ ]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '0': 9,
 '1': 10,
 '2': 11,
 '3': 12,
 '4': 13,
 '5': 14,
 '6': 15,
 '7': 16,
 '8': 17,
 '9': 18,
 ':': 19,
 '?': 20,
 'A': 21,
 'B': 22,
 'C': 23,
 'D': 24,
 'E': 25,
 'F': 26,
 'G': 27,
 'H': 28,
 'I': 29,
 'J': 30,
 'K': 31,
 'L': 32,
 'M': 33,
 'N': 34,
 'O': 35,
 'P': 36,
 'Q': 37,
 'R': 38,
 'S': 39,
 'T': 40,
 'U': 41,
 'V': 42,
 'W': 43,
 'Y': 44,
 'a': 45,
 'b': 46,
 'c': 47,
 'd': 48,
 'e': 49,
 'f': 50,
 'g': 51,
 'h': 52,
 'i': 53,
 'j': 54,
 'k': 55,
 'l': 56,
 'm': 57,
 'n': 58,
 'o': 59,
 'p': 60,
 'q': 61,
 'r': 62,
 's': 63,
 't': 64,
 'u': 65,
 'v': 66,
 'w': 67,
 'x': 68,
 'y': 69,
 'z': 70}

In [ ]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '$': 4,
 '%': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '4': 14,
 '5': 15,
 '6': 16,
 '7': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 '?': 21,
 'A': 22,
 'B': 23,
 'C': 24,
 'D': 25,
 'E': 26,
 'F': 27,
 'G': 28,
 'H': 29,
 'I': 30,
 'J': 31,
 'K': 32,
 'L': 33,
 'M': 34,
 'N': 35,
 'O': 36,
 'P': 37,
 'Q': 38,
 'R': 39,
 'S': 40,
 'T': 41,
 'U': 42,
 'V': 43,
 'W': 44,
 'Y': 45,
 'Z': 46,
 'a': 47,
 'b': 48,
 'c': 49,
 'd': 50,
 'e': 51,
 'f': 52,
 'g': 53,
 'h': 54,
 'i': 55,
 'j': 56,
 'k': 57,
 'l': 58,
 'm': 59,
 'n': 60,
 'o': 61,
 'p': 62,
 'q': 63,
 'r': 64,
 's': 65,
 't': 66,
 'u': 67,
 'v': 68,
 'w': 69,
 'x': 70,
 'y': 71,
 'z': 72,
 '\xa0': 73,
 'Ä': 74,
 'Ö': 75,
 'Ü': 76,
 'ß': 77,
 'ä': 78,
 'ö': 79,
 'ü': 80,
 '’': 81,
 '“': 82,
 '„': 83,
 '\u202f': 84}

In [ ]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [ ]:
encoder_input_data.shape

(10000, 15, 71)

In [ ]:
encoder_input_data[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [ ]:
num_encoder_tokens

71

In [ ]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Epoch 1/10
125/125 [==============================] - 2s 16ms/step - loss: 1.4794 - accuracy: 0.6453 - val_loss: 1.3301 - val_accuracy: 0.6354
Epoch 2/10
125/125 [==============================] - 1s 10ms/step - loss: 1.0413 - accuracy: 0.7213 - val_loss: 1.0074 - val_accuracy: 0.7317
Epoch 3/10
125/125 [==============================] - 1s 9ms/step - loss: 0.8464 - accuracy: 0.7674 - val_loss: 0.8715 - val_accuracy: 0.7641
Epoch 4/10
125/125 [==============================] - 1s 10ms/step - loss: 0.7348 - accuracy: 0.7914 - val_loss: 0.7974 - val_accuracy: 0.7767
Epoch 5/10
125/125 [==============================] - 1s 9ms/step - loss: 0.6740 - accuracy: 0.8072 - val_loss: 0.7562 - val_accuracy: 0.7875
Epoch 6/10
125/125 [==============================] - 1s 10ms/step - loss: 0.6279 - accuracy: 0.8185 - val_loss: 0.7274 - val_accuracy: 0.7947
Epoch 7/10
125/125 [==============================] - 1s 10ms/step - loss: 0.5912 - accuracy: 0.8290 - val_loss: 0.6921 - val_accuracy: 0.8018
E

In [ ]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [ ]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Versuchen Sie es an!

-
Input sentence: Hi.
Decoded sentence: Lass Tom halfen!

-
Input sentence: Hi.
Decoded sentence: Lass Tom halfen!

-
Input sentence: Run!
Decoded sentence: Lass mich nicht an!

-
Input sentence: Run.
Decoded sentence: Lass mich nicht an!

-
Input sentence: Wow!
Decoded sentence: War das das geschen?

-
Input sentence: Wow!
Decoded sentence: War das das geschen?

-
Input sentence: Fire!
Decoded sentence: Behalten Sie es auf!

-
Input sentence: Help!
Decoded sentence: Halle Tom!

-
Input sentence: Help!
Decoded sentence: Halle Tom!

-
Input sentence: Stop!
Decoded sentence: Halt dich auf!

-
Input sentence: Wait!
Decoded sentence: Warte Tom!

-
Input sentence: Wait.
Decoded sentence: Warte Tom!

-
Input sentence: Begin.
Decoded sentence: Versuchen Sie es an!

-
Input sentence: Go on.
Decoded sentence: Geh nicht an!

-
Input sentence: Hello!
Decoded sentence: Halle Tom!

-
Input sentence: Hurry!
Decoded sentence: Lass dich nic